In [ ]:
from vpl_extraction.vpl_star import VPLStar
from examples.dyck1 import Dyck1

In [ ]:
dyck1_grammar = Dyck1()

vpl_star = VPLStar(dyck1_grammar)

In [ ]:
result = vpl_star.learn()

In [ ]:
print(str(result.tree_automaton))

In [ ]:
from models.vpg import vpg_from_tree_automata

vpg = vpg_from_tree_automata(result.tree_automaton, dyck1_grammar.alphabet)

print(vpg.print_grammar())

In [ ]:
print(len(result.tree_automaton.states))

In [ ]:
for k, v in result.tree_automaton.transitions.items():
    print(f"{k} - Val {v}")

In [ ]:
random_word_count = 10000
counterexample = False
for _ in range(random_word_count):
    word = dyck1_grammar.get_random_word()
    
    if result.is_accepted(word) != dyck1_grammar.is_accepted(word):
        print(f"VPLStar does not match Dyck1 for the word: {word}")
        counterexample = True
        break

if not counterexample:
    print("VPLStar matches Dyck1 for all tested words.")

In [ ]:
alphabet = Dyck1().alphabet

In [ ]:
from experiments.transformer_checker_wrapper import TransformerWrapper

transformer = TransformerWrapper(
    metadata_path='./experiments/models/metadata/transformer_model.json',
    alphabet=alphabet
)

In [ ]:
from vpl_extraction.vpl_star_random_comparator import VPLRandomComparator
comparator = VPLRandomComparator(
    alphabet=alphabet,
    random_trees=30,
    max_depth=3
)

In [ ]:
vpl_star = VPLStar(transformer, comparator)

In [ ]:
res = vpl_star.learn()

In [ ]:
vpg = vpg_from_tree_automata(res.tree_automaton, alphabet)

print(vpg.print_grammar())